Optimizing the Knapsack: 
 A Genetic Algorithm Approach

# Section Zero: Overview of Inputs, Libraries, and Necessary Data

## Import libraries:

In [ ]:
import pandas as pd
import numpy as np
import random as random
import matplotlib.pyplot as plt

## Get inputs:

In [ ]:
min_v = float(input("Enter the minimum value (min_v) you want to achieve: "))
max_w = float(input("Enter the maximum weight (max_w) allowed: "))

min_n = int(input("Enter the minimum number of snack types (min_n): "))
max_n = int(input("Enter the maximum number of snack types (max_n): "))

In [ ]:
population_size = 200 #if it shouldn't be considered as an input!

In [ ]:
population_size = int(input("Enter population size: "))

In [ ]:
max_run = 1000 #if it shouldn't be considered as an input!

In [ ]:
max_run = int(input())

In [ ]:
print(f"min value: {min_v},\nmax weight: {max_w},\nrange: {min_n}-{max_n},\npopulation size: {population_size},\nmax run: {max_run}")

## read CSV file and save to Data Frame:

In [ ]:
file_path = 'snacks.csv'
df = pd.read_csv(file_path)

In [ ]:
print(df)

# Part One: Defining Basic Concepts

## Gene:

In [ ]:
class Gene:
    def __init__(self, name, max_weight, value_per_weight, weight = 0):
        self.name = name
        self.weight = weight
        self.max_weight = max_weight
        self.value_per_weight = value_per_weight
          
    def __repr__(self):
        return f"Gene(name={self.name}, weight={self.weight}, max_weight={self.max_weight}, value_per_weight={self.value_per_weight}\n)"

    def __str__(self):
        return f"{self.name}: Weight={self.weight}, Value per Weight={self.value_per_weight}"


## Chromosome:

In [ ]:
class Chromosome:
    def __init__(self, geness, new_born = False):
        self.genes = []
        for g in geness:
            self.genes.append(Gene(g.name, g.max_weight, g.value_per_weight, g.weight))
        if (new_born == True):
            for gene in self.genes:
                while gene.weight == 0:
                    gene.weight = random.uniform(0, gene.max_weight)
        self.total_weight = sum(gene.weight for gene in self.genes)
        self.total_value = sum(gene.weight * gene.value_per_weight for gene in self.genes)
        self.variety_of_snacks = len(self.genes)
        self.fitness = 0 
         
    
    def __repr__(self):
        return f"\nChromosome(genes=\n{self.genes}, fitness={self.fitness}) \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n{"---"*25} \n"
        
    def __str__(self):
        genes_str = '\n'.join(str(gene) for gene in self.genes)
        return f"\nChromosome Details:\nGenes:\n{genes_str}\nFitness: {self.fitness} \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n{"---"*25} \n"
        



## Genes pool:

In [ ]:

genes_pool = df.copy()
genes_pool.rename(columns={'Snack': 'Name'}, inplace=True)
genes_pool.rename(columns={'Available Weight': 'Maximum Weight'}, inplace=True)
genes_pool['Value per Weight'] = genes_pool['Value'] / genes_pool['Maximum Weight']
genes_pool = genes_pool[['Name', 'Maximum Weight', 'Value per Weight']]


In [ ]:
genes_pool

In [ ]:
gene_objects = [Gene(row['Name'], row['Maximum Weight'], row['Value per Weight']) for index, row in genes_pool.iterrows()]

In [ ]:
gene_objects

# Part Two: Primary population production

In [ ]:
initial_population = []
for _ in range(population_size):
    x = random.randint(min_n, max_n)
    genes_temp = random.sample(gene_objects, x)
    c = Chromosome(genes_temp, True)
    initial_population.append(c)
    c = None


In [ ]:
initial_population

# Part three: Implementation and specification of compatibility criterion function

## fitness

In [ ]:
def calculate_fitness(chromosome):
    fitness = chromosome.total_value
    penalty_weight =max_w - chromosome.total_weight
    penalties = 0
    if penalty_weight < 0:
        penalties = penalty_weight 
    if penalties < 0 :
        fitness = penalties
    return fitness
    


#### Update Chromosomes Fitness'

In [ ]:
def update_fitness(population):
    for chromosome in population:
        chromosome.fitness = calculate_fitness(chromosome)
    return population
    

In [ ]:
initial_population = update_fitness(initial_population)

In [ ]:
initial_population

## Population compatibility:

In [ ]:
def print_population_compatibility(population):
    print("Population compatibility:")
    print(f"Avg Fitness: {sum(c.fitness for c in population) / len(population)}")
    print(f"Max Fitness: {max(c.fitness for c in population)}")
    print(f"Avg Value per Weight: {sum(c.total_value / c.total_weight for c in population) / len(population)}")
    print(f"Max Avg Value per Weight: {max(c.total_value / c.total_weight for c in population)}")
    print("--" * 25)

In [ ]:
print_population_compatibility(initial_population)

## find winner

In [ ]:
def find_winner(population):
    max_fitness = float('-inf')
    winner = population[0];
    for chromosome in population:
        if (chromosome.fitness > max_fitness):
            max_fitness = chromosome.fitness
            winner = chromosome
    return winner

In [ ]:
def check_for_answer(population):
    winner = find_winner(population)
    if winner.fitness >= min_v:
        return winner
    return None

In [ ]:
def print_winner(winner):
    for Gene in winner.genes:
        print(f"{Gene.name}: {Gene.weight}")
    print(f"Total Weight: {winner.total_weight}")
    print(f"Total Value: {winner.total_value}")

# Part four: Generating a new generation

#### Probability:

In [ ]:
def decide_with_probability(p, thing1, thing2):
    if random.random() < p:
        return thing1
    else:
        return thing2

## Crossover:

#### Crossover Function:

In [ ]:
def crossover(prob , parent1, parent2, min_n, max_n):
    x = min(len(parent1.genes), len(parent2.genes))
    if x <= 1:
        return [parent1, parent2]
    if parent1 == parent2:
        return [parent1, parent2]
        
    child1 = Chromosome(parent1.genes, False);
    child2 = Chromosome(parent2.genes, False);

    possible1 = [g for g in child1.genes if g.name not in [k.name for k in child2.genes]]
    possible2 = [g for g in child2.genes if g.name not in [k.name for k in child1.genes]]
    if min(len(possible1), len(possible2)) >= 1 :
        i = random.randint(0, min(len(possible1), len(possible2)) - 1)
        child1.genes.remove(possible1[i])
        child2.genes.remove(possible2[i])
        child1.genes.append(possible2[i])
        child2.genes.append(possible1[i])

    def select_new_generation(prob, child1, child2, parent1, parent2):
        if (max_w - child1.total_weight - child2.total_weight > max_w - parent1.total_weight - parent2.total_weight):    
            return decide_with_probability(prob,  [parent1, parent2],  [child1, child2])            
        return decide_with_probability(prob, [child1, child2], [parent1, parent2])

    return select_new_generation(prob, child1, child2, parent1, parent2)
    

In [ ]:
def generate_new_population_crossover(prob, population):
    np.random.shuffle(population)
    new_population = []
    for i  in range(0, round((population_size)/2)):
        parent1 = population[i]
        parent2 = population[-i]
        new_generation = crossover(prob , parent1, parent2, min_n, max_n)
        new_population.append(new_generation[0])
        new_population.append(new_generation[1])
    return new_population

## Mutation:

In [ ]:
def mutation(prob_m, population):
    np.random.shuffle(population)
    new_population = []
    for i  in range(0, population_size - 1):
        parent = population[i]
        genes = []
        for g in parent.genes:
            genes.append(g)

        genes.sort(key=lambda x: x.value_per_weight)

        pre_gene = genes.pop(0)

        sorted_genes = gene_objects
        sorted_genes.sort(key=lambda x: x.value_per_weight, reverse=True)

        new_gene = pre_gene

        flag1 = False 
        flag2 = True
        p = False
        for g in sorted_genes:
            if pre_gene.name == g.name:    
                p = True
                genes.append(pre_gene)
                break
            if any(x.name == g.name for x in genes) == True:
                continue
            else:
                new_gene = g
                break
        if p: 
            n = random.randint(0, len(genes) - 1)
            if (parent.total_weight > max_w):
                genes[n].weight = random.uniform(0, genes[n].weight)
                while genes[n].weight == 0:
                    genes[n].weight = random.uniform(0, genes[n].weight)
            else:
                genes[n].weight = random.uniform(genes[n].weight, genes[n].max_weight)
            
        else:
            if parent.variety_of_snacks > min_n:
                flag1 = True 
            if pre_gene.weight == pre_gene.max_weight: 
                flag2 = True 
            if flag1:
                if random.random() > 0.1:
                    genes.append(new_gene)
                else:
                    if len(genes) < min_n:
                         genes.append(new_gene)
            else: 
                genes.append(new_gene)
            if flag2:
                not_in_parent = [obj for obj in sorted_genes if obj.name not in [objj.name for objj in parent.genes]]
                if random.random() > 0.9 and len(not_in_parent) != 0:
                    new_new = random.choice(not_in_parent)
                    new_new.weight = random.uniform(0, new_new.max_weight)
                    if any(x.name == new_new.name for x in genes) == False and len(genes) < max_n:
                        genes.append(new_new)
            for g in genes:
                if g.name == new_gene.name:
                    g.weight = random.uniform(min(g.max_weight, pre_gene.weight), new_gene.max_weight)
                    break               
        if len(genes) == 0:
            return population
        child = Chromosome(genes, False)
        #print(f"parent: {parent}")
        #print (f"child: {child}")
        #print("__" * 10)
        genes.clear() 
        new_generation = decide_with_probability(prob_m, parent, child)
        new_population.append(new_generation)
    return new_population

## Generating a new generation

In [ ]:
def generate_new_geneation(population, prob_c, prob_m):
        new_population = population
        new_population = generate_new_population_crossover(prob_c, new_population)
        new_population = mutation(prob_m, new_population)
        new_population = update_fitness(new_population)
        return new_population

# Part five: Creating a Genetic Algorithm on the Problem

## Algorithm:

In [ ]:
def genetic_algorithm(initial_population):
    initial_population = update_fitness(initial_population)
    population = initial_population
    winner = check_for_answer(population)
    counter = 0
    cur_fit = sum(c.fitness for c in population) / len(population)
    value_per_weight = sum(c.total_value / c.total_weight for c in population) / len(population)
    
    while winner == None and counter < max_run:
        fitness_arr.append(cur_fit)
        val_weight_arr.append(value_per_weight)   
        max_fitness_arr.append(max(c.fitness for c in population))
        max_val_weight_arr.append(max(c.total_value / c.total_weight for c in population)) 
        counter = counter + 1
        prob_m = max (1 / (counter + 1), 1/10)
        prob_c = 0.25
        population = generate_new_geneation(population, prob_c, prob_m)
        cur_fit = sum(c.fitness for c in population)/ len(population)
        value_per_weight = sum(c.total_value / c.total_weight for c in population) / len(population)
        winner =check_for_answer(population)
        
    fitness_arr.append(cur_fit)
    val_weight_arr.append(value_per_weight)   
    max_fitness_arr.append(max(c.fitness for c in population))
    max_val_weight_arr.append(max(c.total_value / c.total_weight for c in population)) 
        
    if winner != None:
        print(f"OK -#run: {counter + 1}")
    else:
        print("fail")
    return winner, population

# Part six: Evaluation of Results

## Execute Program:

In [ ]:
fitness_arr = []
val_weight_arr = []
max_fitness_arr = []
max_val_weight_arr = []
winner, population = genetic_algorithm(initial_population)

In [ ]:
if winner != None:
    print_winner(winner)
else: 
    print("No answer found:\n")
    
    print("The best possible answer: ")
    print(find_winner(population))

## Details:

In [ ]:
def plot_details(val_weight_arr, fitness_arr, max_val_weight_arr, max_fitness_arr):
    def plot_detail(y, title, axes, i):
        x = [i for i in range(0, len(y))]
        axes[i].set_title(title)
        axes[i].plot(x,y)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    plot_detail(fitness_arr, "Avg Fitness", axes, 0)
    plot_detail(val_weight_arr, "Avg Value Per Weight", axes, 1)
    fig.tight_layout()
    plt.show()
    fig2, axes2 = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    plot_detail(max_fitness_arr, "Max Fitness", axes2, 0)
    plot_detail(max_val_weight_arr, "Max Value Per Weight", axes2, 1)
    fig2.tight_layout()
    plt.show()

In [ ]:
print(f"#run = {len(max_fitness_arr)}")
plot_details(val_weight_arr, fitness_arr, max_val_weight_arr, max_fitness_arr)


In [ ]:
def print_details(val_weight_arr, fitness_arr, max_val_weight_arr, max_fitness_arr):
    for i in range(0, len(fitness_arr)):
        print(f"i: {i}")
        print(f"avg fitness: {fitness_arr[i]}")
        print(f"max fitness: {max_fitness_arr[i]}")
        print(f"avg value per weight: {val_weight_arr[i]}")
        print(f"max value per weight: {max_val_weight_arr[i]}")
        print("_" * 50)

In [ ]:
print_details(val_weight_arr, fitness_arr,  max_val_weight_arr, max_fitness_arr)